from Ankur -- helpers for loading csv from s3 into memory and for pushing results to s3 

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [14]:
import boto3
import pandas as pd
import os

s3_client = boto3.client('s3')

### 1. Get a list of available buckets in the environment

In [3]:
def list_buckets():
    for buckets in s3_client.list_buckets()['Buckets']:
        print(buckets['Name'])

In [10]:
list_buckets()

aws-glue-scripts-577076019476-us-east-1
aws-glue-temporary-577076019476-us-east-1
aws-logs-577076019476-us-east-1
fhwa-carma-fotda-dev-cfdeploy
fotda-preprocessed
fotda-raw
fotda-s3-bucket
fotda-src
pre-processed-fotda-cav-modeling
preprocessed-carma-core-validation
raw-carma-core-validation
raw-fotda-cav-modeling


### 2. Get a list of all files within a specific bucket and directory path

In [9]:
def list_files_from_bucket(bucket, key):
    response = s3_client.list_objects(Bucket=bucket, Prefix=key)
    s3_objects = response['Contents']
#     s3_uri_prefix = f's3://{bucket}'

    for obj in s3_objects:
        filename = obj['Key'] #f'{s3_uri_prefix}/{obj["Key"]}' #((obj['Key']).split('/'))[-1]
        print(filename)

In [10]:
bucket = 'preprocessed-carma-core-validation'
key = 'csvfiles/Core_Validation_Testing/Facility_Summit_Point/Vehicle_Black_Pacifica/20210318/r1_down-selected/'
list_files_from_bucket(bucket, key)

csvfiles/Core_Validation_Testing/Facility_Summit_Point/Vehicle_Black_Pacifica/20210318/r1_down-selected/
csvfiles/Core_Validation_Testing/Facility_Summit_Point/Vehicle_Black_Pacifica/20210318/r1_down-selected/carma_record_rosparams.csv
csvfiles/Core_Validation_Testing/Facility_Summit_Point/Vehicle_Black_Pacifica/20210318/r1_down-selected/carma_system_version.csv
csvfiles/Core_Validation_Testing/Facility_Summit_Point/Vehicle_Black_Pacifica/20210318/r1_down-selected/cluster_centroids.csv
csvfiles/Core_Validation_Testing/Facility_Summit_Point/Vehicle_Black_Pacifica/20210318/r1_down-selected/detection_fusion_tools_objects.csv
csvfiles/Core_Validation_Testing/Facility_Summit_Point/Vehicle_Black_Pacifica/20210318/r1_down-selected/detection_image_detector_objects.csv
csvfiles/Core_Validation_Testing/Facility_Summit_Point/Vehicle_Black_Pacifica/20210318/r1_down-selected/detection_lidar_detector_objects.csv
csvfiles/Core_Validation_Testing/Facility_Summit_Point/Vehicle_Black_Pacifica/20210318/r

### 3. Import and load a csv file from S3 as pandas dataframe for further analysis

In [11]:
file_name = "csvfiles/Core_Validation_Testing/Facility_Summit_Point/Vehicle_Black_Pacifica/20210318/r1_down-selected/hardware_interface_gps.csv"
obj = s3_client.get_object(Bucket=bucket, Key=file_name)
df = pd.read_csv(obj['Body'])

In [8]:
df.tail()

rosbagTimestamp  header   seq  stamp        secs      nsecs  \
3166  1616001691142760328     NaN  3881    NaN  1615962331  132123772   
3167  1616001691169042476     NaN  3882    NaN  1615962331  150420307   
3168  1616001691227908616     NaN  3883    NaN  1615962331  223609075   
3169  1616001691254512285     NaN  3884    NaN  1615962331  248298834   
3170  1616001691290042559     NaN  3885    NaN  1615962331  281937172   

            frame_id  status  header.1  seq.1  ...  err_vert  err_track  \
3166  "novatel_gnss"     NaN       NaN      0  ...  1.085184        0.0   
3167  "novatel_gnss"     NaN       NaN      0  ...  1.085184        0.0   
3168  "novatel_gnss"     NaN       NaN      0  ...  1.085184        0.0   
3169  "novatel_gnss"     NaN       NaN      0  ...  1.085184        0.0   
3170  "novatel_gnss"     NaN       NaN      0  ...  1.085184        0.0   

      err_speed err_climb  err_time err_pitch  err_roll err_dip  \
3166        0.0       0.0       0.0       0.0       0.0     0.0   
3167        0.0       0.0       0.0       0.0       0.0     0.0   
3168        0.0       0.0       0.0       0.0       0.0     0.0   
3169        0.0       0.0       0.0       0.0       0.0     0.0   
3170        0.0       0.0       0.0       0.0       0.0     0.0   

                                    position_covariance  \
3166  [0.1479666673219846, 0.0, 0.0, 0.0, 0.19790024...   
3167  [0.1479666673219846, 0.0, 0.0, 0.0, 0.19790024...   
3168  [0.1479666673219846, 0.0, 0.0, 0.0, 0.19790024...   
3169  [0.1479666673219846, 0.0, 0.0, 0.0, 0.19790024...   
3170  [0.1479666673219846, 0.0, 0.0, 0.0, 0.19790024...   

     position_covariance_type  
3166                        2  
3167                        2  
3168                        2  
3169                        2  
3170                        2  

[5 rows x 52 columns]

In [9]:
df.columns.values

array(['rosbagTimestamp', 'header', 'seq', 'stamp', 'secs', 'nsecs',
       'frame_id', 'status', 'header.1', 'seq.1', 'stamp.1', 'secs.1',
       'nsecs.1', 'frame_id.1', 'satellites_used', 'satellite_used_prn',
       'satellites_visible', 'satellite_visible_prn',
       'satellite_visible_z', 'satellite_visible_azimuth',
       'satellite_visible_snr', 'status.1', 'motion_source',
       'orientation_source', 'position_source', 'latitude', 'longitude',
       'altitude', 'track', 'speed', 'climb', 'pitch', 'roll', 'dip',
       'time', 'gdop', 'pdop', 'hdop', 'vdop', 'tdop', 'err', 'err_horz',
       'err_vert', 'err_track', 'err_speed', 'err_climb', 'err_time',
       'err_pitch', 'err_roll', 'err_dip', 'position_covariance',
       'position_covariance_type'], dtype=object)

### 4. Download a file from S3

In [6]:
def download_from_s3(s3_filename, local_filename):
    return os.system(f'aws s3 cp {s3_filename} {local_filename}')

In [17]:
s3_filename = f's3://{bucket}/{file_name}' #bucket variable is from Step2 and filename variable is from step3 above
local_filename = 'C:\\Users\\zankur.tyagi.ctr\\Documents\\Core_Validation\\bagfiles' #change this path to your local user account
download_from_s3(s3_filename, local_filename)

0

### 5. Upload file to S3

In [16]:
def upload_to_s3(local_filename, s3_filename):
    return os.system(f'aws s3 cp {local_filename} {s3_filename}')

In [18]:
local_filename = 'C:\\Users\\zankur.tyagi.ctr\\Documents\\Core_Validation\\bagfiles\\hardware_interface_gps.csv'
s3_filename = 's3://fotda-raw/bagfiles/Core_Validation_Testing/Facility_Summit_Point/logs/'
upload_to_s3(local_filename, s3_filename)

0